In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [3]:
abund_tb=pd.read_csv("../data/LSU/BFTMS23071402 (single)OTU (Table).csv")
trim_tbl=pd.read_csv("../data/Trim_summary.csv")

In [4]:
abund_tb.head(5)

,ID,Name,Taxonomy,Combined Abundance,Min,Max,Mean,Median,Std,BFTMS23071402 Abundance,...,BFTMS23071409 Abundance,BFTMS23071410 Abundance,BFTMS23071411 Abundance,BFTMS23071412 Abundance,BFTMS23071413 Abundance,BFTMS23071414 Abundance,BFTMS23071415 Abundance,BFTMS23071416 Abundance,BFTMS23071417 Abundance,Sequence
0,Synthetic construct; FB985460.2.2274,FB985460.2.2274,Eukaryota; Ascomycota; Saccharomycetes; Saccha...,12,0,3,0.7500,0.5,0.930949,1,...,1,1,0,2,0,0,0,0,3,GAAAAGAACTTTGAAAAGAGAGTGAAAAAGTACGTGAAATTGTTGA...
1,Cocos nucifera (coconut palm); GABT01019965.57...,GABT01019965.570.3684,Bacteria; Cyanobacteria; Cyanobacteriia; Chlor...,3,0,3,0.1875,0.0,0.750000,0,...,0,0,0,0,0,0,0,0,0,AACGCGTAAGAACCTGCCCTTGGGAGGGGAACAACAACTGGAAACG...
2,Polistes fuscatus (common paper wasp); GDFS011...,GDFS01112734.1.2535,Eukaryota; Arthropoda; Insecta; ; ; ; Polistes...,4,0,2,0.2500,0.0,0.577350,1,...,0,0,0,0,0,0,0,0,0,GGAAAGAGAGTTAAAAAGCACGTGAAATTGTTAAAAGGGAAGGGAT...
3,Mangifera indica (mango); GBJO01000463.79.3249,GBJO01000463.79.3249,Eukaryota; Ascomycota; Dothideomycetes; Capnod...,11,0,3,0.6875,0.0,0.946485,1,...,0,0,0,2,0,0,0,0,3,GGAAAGAGAGTTAAAAAGCACGTGAAATTGTTAAAAGGGAAGGGAT...
4,Staphylococcus aureus M1060; AQGA01000005.4140...,AQGA01000005.41407.43923,Bacteria; Firmicutes; Bacilli; Staphylococcale...,96,0,53,6.0000,0.0,16.325849,0,...,0,0,0,0,0,0,0,0,0,GCAGGGTCGAAAGACGGACTTAGTGTTCCTTGGGCAGCGAAACCGC...


In [5]:
# Rename the columns by removing the keyword abundance and BFTMS
abund_tb.columns = [col.replace("Abundance", '') for col in abund_tb.columns]
abund_tb.columns = [col.replace(" ", '') for col in abund_tb.columns]
#abund_tb.columns = [col.replace("BFTMS", '') for col in abund_tb.columns]

##add a control column that gets values from the spcified control sample
control="BFTMS23071417"
#abund_tb["control"]=abund_tb[control]

samples=trim_tbl["Name"].astype(str).tolist()
#samples.append('control')

abund_tb['reference_lib']='LSU'

In [6]:
## mark species with high risk of false positive detection in a column "false_pos_prone"  

# first split the ID column into species group name and species ID
abund_tb[['species group', 'species id']] = abund_tb['ID'].str.split(';', expand=True)
## create a new column "false_pos_prone" and mark species with high risk of false positive detection
abund_tb['false_pos_prone']='No'


def mark_species(df, ID_column, false_pos_tag, species_list):
    # Update Column B to True for rows where values in Column A start with any of the strings in the search list
    for name in species_list:
        df.loc[df[ID_column].str.startswith(name), false_pos_tag] = 'Yes'
    return df


false_pos_prone_spicies=['Enterobacter aerogenes', 'Saccharomyces cerevisiae', 'Streptococcus pneumoniae','Klebsiella pneumoniae', 
                         'Serratia marcescens', 'Salmonella enterica']
abund_tb = mark_species(abund_tb, 'species group', 'false_pos_prone', false_pos_prone_spicies)
#abund_tb.loc[abund_tb['species group'].contains(false_pos_prone_spicies), 'false_pos_prone'] = 'Yes'

In [7]:
## data normalization and corection for control
control="BFTMS23071417"
resid_val=0.00 # this value will be added to sample and control to avoid deviding to 0

# normalize all abundances based on the total reads for each sample
for samp in samples:
    ## read the trimmed reads from trim_tbl and get the value based on the sample name
    trimmed_reads=trim_tbl[trim_tbl["Name"]==samp]["Trimmed sequences"].values[0]
    abund_tb[samp+"_Normalized"]=abund_tb[samp]*trimmed_reads/1000000

## correct normized abundance by deducing control abundance
for samp in samples:
    abund_tb[samp+"_Normalized_corrected"]=abund_tb[samp+"_Normalized"]-abund_tb[control+"_Normalized"]
    ## calculate signal to noize 
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

## ## calculate signal to noize
for samp in samples:
    abund_tb[samp+"_S:N"]=(abund_tb[samp+"_Normalized_corrected"]+resid_val)/(abund_tb[control+"_Normalized"]+resid_val)

In [8]:
## check one sample and compare it against the results from manual analysis
sample='BFTMS23071405'
sample_df=abund_tb[['ID', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']]
sample_df.sort_values(by=sample+'_Normalized_corrected', ascending=False)
sample_df.head(5)

,ID,BFTMS23071405,BFTMS23071417,BFTMS23071405_Normalized,BFTMS23071417_Normalized,BFTMS23071405_Normalized_corrected,BFTMS23071405_S:N
0,Synthetic construct; FB985460.2.2274,2,3,0.346402,0.302109,0.044293,0.146613
1,Cocos nucifera (coconut palm); GABT01019965.57...,0,0,0.000000,0.000000,0.000000,NaN
2,Polistes fuscatus (common paper wasp); GDFS011...,2,0,0.346402,0.000000,0.346402,inf
3,Mangifera indica (mango); GBJO01000463.79.3249,1,3,0.173201,0.302109,-0.128908,-0.426694
4,Staphylococcus aureus M1060; AQGA01000005.4140...,0,0,0.000000,0.000000,0.000000,NaN


In [9]:
## LSU strict

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=150) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=50) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0))
                            ]
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='LSU_strict'
select_IDs.to_csv('../output_tables/LSU_strict.csv')




In [10]:
## LSU moderate

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    

    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=60) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=20) & 
                            ((sample_abund['control_abund']<=3) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0))
                            ]

    
    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='LSU_moderate'
select_IDs.to_csv('../output_tables/LSU_moderate.csv')




In [11]:
## LSU loose

standard_col_names=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', 'sample_abund', 'control_abund','sample_normalized', 'control_normalized','normalized_corrected', 'S:N', 'sample', 'control']
select_IDs=pd.DataFrame(columns=standard_col_names)

for sample in samples:
    select_columns=['ID','Name','Taxonomy','species group', 'species id','reference_lib','false_pos_prone','Combined','Min','Max','Mean','Median','Std', sample, control,sample+'_Normalized', control+'_Normalized',sample+'_Normalized_corrected', sample+'_S:N']
    sample_abund=abund_tb[select_columns]
    sample_abund['sample']=sample
    sample_abund['control']=control
    sample_abund.columns=standard_col_names
    #sample_abund.rename(columns={sample: "sample",control:"control", sample+'_Normalized':"Normalized'}
    sample_abund=sample_abund.sort_values(by='normalized_corrected', ascending=False)
    sample_abund=sample_abund[0:15]
    
    select_ID=sample_abund[((sample_abund['false_pos_prone']=='Yes') &
                            (sample_abund['sample_abund']>=15) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=1000)) &
                            (sample_abund['normalized_corrected']>=0)) 
                            |
                            ((sample_abund['false_pos_prone']=='No') &
                            (sample_abund['sample_abund']>=5) & 
                            ((sample_abund['control_abund']==0) | (sample_abund['S:N']>=3)) &
                            (sample_abund['normalized_corrected']>=3))
                            ]

    select_IDs=pd.concat([select_IDs, select_ID], axis=0)

select_IDs['mode']='LSU_loose'
select_IDs.to_csv('../output_tables/LSU_loose.csv')


